In [1]:
# 导入必要的库
import pandas as pd
import numpy as np

# 读取CSV文件
df = pd.read_csv('charging_station_data.csv')
df = df.reset_index(drop=True)
# 显示数据的基本信息
print(df.info())

# 检查tariff_amount列的空值数量
print("\n空值数量:")
print(df['tariff_amount'].isnull().sum())

# 定义一个函数来填充tariff_amount的空值
def fill_tariff_amount(group):
    mask = group['tariff_amount'].isnull()
    group.loc[mask, 'tariff_amount'] = group['tariff_amount'].mean()
    return group

# 按max_chargerate, plug_type, connector_type分组，并应用填充函数
df = df.groupby(['max_chargerate', 'plug_type', 'connector_type'], group_keys=False).apply(fill_tariff_amount)

# 再次检查tariff_amount列的空值数量
print("\n填充后的空值数量:")
print(df['tariff_amount'].isnull().sum())

# 显示填充后的数据样本
print("\n填充后的数据样本:")
print(df[['max_chargerate', 'plug_type', 'connector_type', 'tariff_amount']].sample(10))

# 检查填充是否正确
print("\n每组的平均tariff_amount:")
print(df.groupby(['max_chargerate', 'plug_type', 'connector_type'])['tariff_amount'].mean())

/var/folders/65/gwq9096x7_dbpj8wqbnt8gy00000gn/T/ipykernel_1915/2155779481.py:6: DtypeWarning: Columns (1,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('charging_station_data.csv')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3011648 entries, 0 to 3011647
Data columns (total 14 columns):
 #   Column                Dtype  
---  ------                -----  
 0   city_id               int64  
 1   station_name          object 
 2   connector_id          int64  
 3   coordinates_x         float64
 4   coordinates_y         float64
 5   postcode              object 
 6   tariff_amount         float64
 7   tariff_connectionfee  float64
 8   max_chargerate        object 
 9   plug_type             object 
 10  connector_type        object 
 11  datetime              object 
 12  weather               object 
 13  is_available          int64  
dtypes: float64(4), int64(3), object(7)
memory usage: 321.7+ MB
None

空值数量:
97707


/var/folders/65/gwq9096x7_dbpj8wqbnt8gy00000gn/T/ipykernel_1915/2155779481.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(['max_chargerate', 'plug_type', 'connector_type'], group_keys=False).apply(fill_tariff_amount)



填充后的空值数量:
0

填充后的数据样本:
        max_chargerate    plug_type connector_type  tariff_amount
1974950             50          ccs          Rapid       0.700000
2160941             50      chademo          Rapid       0.500000
87658                7  type_2_plug             AC       0.414937
2961142             22  type_2_plug             AC       0.550000
1637838             22  type_2_plug             AC       0.450000
2201833              7  type_2_plug             AC       0.450000
211542              22  type_2_plug             AC       0.400000
1410315             43  type_2_plug          Rapid       0.550000
348378               7  type_2_plug             AC       0.500000
1616675             51      chademo          Rapid       0.500000

每组的平均tariff_amount:
max_chargerate  plug_type    connector_type        
0               type_2_plug  AC                        0.210000
7               UNKNOWN      Rapid                     0.500000
                type_2_plug  AC                  

In [2]:
# 检查 tariff_connectionfee 列的空值数量
print("\ntariff_connectionfee 列的空值数量(填充前):")
print(df['tariff_connectionfee'].isnull().sum())

# 将 tariff_connectionfee 列的空值填充为 0
df['tariff_connectionfee'] = df['tariff_connectionfee'].fillna(0)

# 再次检查 tariff_connectionfee 列的空值数量,确保填充成功
print("\ntariff_connectionfee 列的空值数量(填充后):")
print(df['tariff_connectionfee'].isnull().sum())

# 显示填充后的数据样本
print("\n填充后的数据样本:")
print(df[['station_name', 'tariff_amount', 'tariff_connectionfee']].sample(10))

# 检查 tariff_connectionfee 列的基本统计信息
print("\ntariff_connectionfee 列的基本统计信息:")
print(df['tariff_connectionfee'].describe())


tariff_connectionfee 列的空值数量(填充前):
1902719

tariff_connectionfee 列的空值数量(填充后):
0

填充后的数据样本:
        station_name  tariff_amount  tariff_connectionfee
131524         50551           0.60                   1.0
948988         52063           0.69                   0.0
1506224        53243           0.50                   0.0
2746878        61753           0.35                   1.0
1527066        53262           0.70                   0.0
2450687        61079           0.40                   0.0
2191567        60510           0.55                   0.0
1218228        52508           0.43                   0.0
3003117        70132           0.15                   0.0
769665         51721           0.47                   0.0

tariff_connectionfee 列的基本统计信息:
count    3.011648e+06
mean     1.247873e-01
std      3.472551e-01
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      0.000000e+00
max      2.000000e+00
Name: tariff_connectionfee, dtype: float64


In [3]:
# 显示原始数据的行数
print(f"原始数据行数: {len(df)}")

# 检查每列中 'UNKNOWN' 的数量
columns_to_check = ['max_chargerate', 'plug_type', 'connector_type']
for col in columns_to_check:
    unknown_count = (df[col] == 'UNKNOWN').sum()
    print(f"{col} 中 'UNKNOWN' 的数量: {unknown_count}")

# 删除包含 'UNKNOWN' 的行
df_cleaned = df[~df[columns_to_check].isin(['UNKNOWN']).any(axis=1)]

# 显示清理后的数据行数
print(f"\n清理后的数据行数: {len(df_cleaned)}")

# 再次检查是否还有 'UNKNOWN' 值
for col in columns_to_check:
    unknown_count = (df_cleaned[col] == 'UNKNOWN').sum()
    print(f"{col} 中剩余 'UNKNOWN' 的数量: {unknown_count}")

# 显示清理后的数据样本
print("\n清理后的数据样本:")
print(df_cleaned[columns_to_check + ['tariff_amount']].sample(10))

# 将 max_chargerate 转换为数值类型（如果还不是的话）
df_cleaned['max_chargerate'] = pd.to_numeric(df_cleaned['max_chargerate'], errors='coerce')

# 显示清理后数据的基本统计信息
print("\n清理后数据的基本统计信息:")
print(df_cleaned[columns_to_check + ['tariff_amount']].describe())

# 更新原始 DataFrame
df = df_cleaned.copy()

原始数据行数: 3011648
max_chargerate 中 'UNKNOWN' 的数量: 979
plug_type 中 'UNKNOWN' 的数量: 19327
connector_type 中 'UNKNOWN' 的数量: 978

清理后的数据行数: 2991343
max_chargerate 中剩余 'UNKNOWN' 的数量: 0
plug_type 中剩余 'UNKNOWN' 的数量: 0
connector_type 中剩余 'UNKNOWN' 的数量: 0

清理后的数据样本:
        max_chargerate    plug_type connector_type  tariff_amount
775886              22  type_2_plug             AC       0.450000
745086              22  type_2_plug             AC       0.500000
2362585              7  type_2_plug             AC       0.450000
1563128             22  type_2_plug             AC       0.400000
43673               22  type_2_plug             AC       0.419101
2243307              7  type_2_plug             AC       0.480000
667624              22  type_2_plug          Rapid       0.150000
2971991             22  type_2_plug             AC       0.550000
267201               7  type_2_plug             AC       0.850000
4104                 7  type_2_plug             AC       0.420000


/var/folders/65/gwq9096x7_dbpj8wqbnt8gy00000gn/T/ipykernel_1915/2060580324.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['max_chargerate'] = pd.to_numeric(df_cleaned['max_chargerate'], errors='coerce')



清理后数据的基本统计信息:
       max_chargerate  tariff_amount
count    2.991343e+06   2.991343e+06
mean     2.324927e+01   4.701428e-01
std      1.585277e+01   4.498947e-01
min      0.000000e+00   0.000000e+00
25%      7.000000e+00   4.000000e-01
50%      2.200000e+01   4.500000e-01
75%      2.200000e+01   5.500000e-01
max      1.500000e+02   3.400000e+01


In [4]:
# 确保 datetime 列是日期时间格式
df['datetime'] = pd.to_datetime(df['datetime'])

# 按 station_name 和 datetime 排序
df = df.sort_values(['station_name', 'datetime'])

# 检查 weather 列的空值数量（处理前）
print("Weather 列空值数量（处理前）:")
print(df['weather'].isnull().sum())

# 定义一个函数来填充 weather 列的空值
def fill_weather(group):
    group['weather'] = group['weather'].fillna(method='ffill')
    group['weather'] = group['weather'].fillna(method='bfill')
    return group

# 对每个充电站分组应用填充函数
df = df.groupby('station_name').apply(fill_weather)

# 重置索引
df = df.reset_index(drop=True)

# 检查 weather 列的空值数量（处理后）
print("\nWeather 列空值数量（处理后）:")
print(df['weather'].isnull().sum())

# 打印索引信息以确认
print("\n索引信息:")
print(df.index)

Weather 列空值数量（处理前）:
493831


/var/folders/65/gwq9096x7_dbpj8wqbnt8gy00000gn/T/ipykernel_1915/1847465488.py:13: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group['weather'] = group['weather'].fillna(method='ffill')
/var/folders/65/gwq9096x7_dbpj8wqbnt8gy00000gn/T/ipykernel_1915/1847465488.py:14: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  group['weather'] = group['weather'].fillna(method='bfill')
/var/folders/65/gwq9096x7_dbpj8wqbnt8gy00000gn/T/ipykernel_1915/1847465488.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('station_


Weather 列空值数量（处理后）:
0

索引信息:
RangeIndex(start=0, stop=2991343, step=1)


In [5]:
# 定义一个函数来去除字符串两端的引号
def remove_quotes(text):
    if isinstance(text, str):
        return text.strip("'\"")
    return text

# 应用函数到 weather 列
df['weather'] = df['weather'].apply(remove_quotes)

# 检查处理结果
print("处理后的 weather 列唯一值：")
print(df['weather'].unique())

# 显示天气种类及其频率
print("\n天气种类及其频率：")
print(df['weather'].value_counts())

# 显示包含引号的天气记录（如果还有的话）
quotes = df[df['weather'].str.contains("'|\"", na=False)]
if not quotes.empty:
    print("\n仍然包含引号的记录：")
    print(quotes[['station_name', 'datetime', 'weather']])
else:
    print("\n所有引号已成功移除。")

# 显示处理后的数据样本
print("\n处理后的数据样本：")
print(df[['station_name', 'datetime', 'weather']].sample(10))

处理后的 weather 列唯一值：
['Rain' 'Clouds' 'Fog' 'Clear' 'Drizzle' 'Mist' 'Haze']

天气种类及其频率：
weather
Clouds     2454997
Rain        330784
Clear       180369
Mist         12628
Drizzle       6734
Haze          4554
Fog           1277
Name: count, dtype: int64

所有引号已成功移除。

处理后的数据样本：
        station_name            datetime weather
2147579        60468 2024-06-23 17:00:00  Clouds
924427         52056 2024-06-20 20:00:00  Clouds
522220         51364 2024-06-28 06:00:00  Clouds
2696193        61707 2024-06-19 17:00:00  Clouds
1423511        53085 2024-06-24 17:00:00  Clouds
1808173        54010 2024-06-16 17:00:00    Rain
901150         52025 2024-06-29 14:00:00  Clouds
505632         51325 2024-07-01 20:00:00  Clouds
1520774        53276 2024-07-01 20:00:00  Clouds
1928132        54324 2024-06-16 02:00:00  Clouds


In [6]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['hour'] = df['datetime'].dt.hour
df['day_of_week'] = df['datetime'].dt.dayofweek
df['is_weekend'] = df['day_of_week'].isin([5, 6]).astype(int)
df['is_holiday'] = ((df['datetime'].dt.dayofweek.isin([5, 6])) | 
                    ((df['datetime'].dt.dayofweek == 4) & (df['hour'] >= 18))).astype(int)
df['time_of_day'] = pd.cut(df['hour'], 
                           bins=[0, 6, 12, 18, 24], 
                           labels=False,
                           include_lowest=True)
df['is_work_hour'] = ((df['day_of_week'].isin([0,1,2,3,4])) & (df['hour'].between(9, 17))).astype(int)

In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns


In [8]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

# 创建 OneHotEncoder 对象
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# 对 'plug_type' 进行独热编码
plug_type_encoded = ohe.fit_transform(df[['plug_type']])
plug_type_columns = [f'plug_type_{cat}' for cat in ohe.categories_[0]]
df_plug_type = pd.DataFrame(plug_type_encoded, columns=plug_type_columns, index=df.index)

# 对 'connector_type' 进行独热编码
connector_type_encoded = ohe.fit_transform(df[['connector_type']])
connector_type_columns = [f'connector_type_{cat}' for cat in ohe.categories_[0]]
df_connector_type = pd.DataFrame(connector_type_encoded, columns=connector_type_columns, index=df.index)

# 对 'weather' 进行独热编码
weather_encoded = ohe.fit_transform(df[['weather']])
weather_columns = [f'weather_{cat}' for cat in ohe.categories_[0]]
df_weather = pd.DataFrame(weather_encoded, columns=weather_columns, index=df.index)

# 将编码后的特征合并到原始数据框
df = pd.concat([df, df_plug_type, df_connector_type, df_weather], axis=1)

In [9]:
# 确保 station_name 和 connector_id 都是字符串类型
df['station_name'] = df['station_name'].astype(str)
df['connector_id'] = df['connector_id'].astype(str)

# 使用 str.cat 方法创建 connector_unique_id
df['connector_unique_id'] = df['station_name'].str.cat(df['connector_id'], sep='_')

In [11]:
df['connector_avg_usage'] = 1 - df.groupby('connector_unique_id')['is_available'].transform('mean')

In [12]:
df['station_avg_usage'] = 1 - df.groupby('station_name')['is_available'].transform('mean')

In [13]:
df = df.sort_values(['station_name', 'connector_id', 'datetime'])

In [14]:
df['usage_last_24h'] = df.groupby('connector_unique_id')['is_available'].transform(
    lambda x: 1 - x.rolling(window=24, min_periods=1).mean()
)

In [15]:
df['usage_last_7d'] = df.groupby('connector_unique_id')['is_available'].transform(
    lambda x: 1 - x.rolling(window=24*7, min_periods=1).mean()
)

In [16]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean

# 假设 df 是你的主数据框
# 如果 df 还没有被定义，你需要先读取你的数据
# 例如：df = pd.read_csv('your_data.csv')

# 1. 到市中心的距离
city_centers = df.groupby('city_id')[['coordinates_x', 'coordinates_y']].mean()

def distance_to_center(row):
    city_center = city_centers.loc[row['city_id']]
    return euclidean((row['coordinates_x'], row['coordinates_y']), 
                     (city_center['coordinates_x'], city_center['coordinates_y']))

df['distance_to_center'] = df.apply(distance_to_center, axis=1)

# 2. 每个城市的充电站密度
stations_per_city = df.groupby('city_id')['station_name'].nunique()

def safe_city_area(group):
    x_range = group['coordinates_x'].max() - group['coordinates_x'].min()
    y_range = group['coordinates_y'].max() - group['coordinates_y'].min()
    area = x_range * y_range
    return max(area, 1e-10)  # 使用一个很小的正数来替代 0

city_areas = df.groupby('city_id').apply(safe_city_area)
city_density = stations_per_city / city_areas
city_density = city_density.replace([np.inf, -np.inf], np.nan)
city_density = city_density.fillna(city_density.mean())

df['city_station_density'] = df['city_id'].map(city_density)

# 3. 使用分位数划分密度等级
def density_to_level_quantile(density, q_dict):
    for level, threshold in sorted(q_dict.items(), key=lambda x: x[1], reverse=True):
        if density >= threshold:
            return level
    return 1  # 如果密度小于所有阈值，返回最低等级

quantiles = city_density.quantile([0.2, 0.4, 0.6, 0.8])
q_dict = {5: quantiles[0.8], 4: quantiles[0.6], 3: quantiles[0.4], 2: quantiles[0.2], 1: 0}

df['city_density_level'] = df['city_id'].map(city_density).apply(lambda x: density_to_level_quantile(x, q_dict))

# 4. 查看结果
print("新特征的前几行：")
print(df[['city_id', 'station_name', 'distance_to_center', 'city_station_density', 'city_density_level']].head(10))

print("\n新特征的统计信息：")
print(df[['distance_to_center', 'city_station_density']].describe())

print("\n密度等级的分布：")
print(df['city_density_level'].value_counts().sort_index())

print("\n每个密度等级的密度范围：")
for level, threshold in sorted(q_dict.items()):
    if level == 5:
        print(f"等级 {level}: >= {threshold:.2f}")
    elif level == 1:
        print(f"等级 {level}: < {q_dict[2]:.2f}")
    else:
        print(f"等级 {level}: {threshold:.2f} - {q_dict[level+1]:.2f}")

# 5. 检查空值
print("\n检查新特征的空值：")
print(df[['distance_to_center', 'city_station_density', 'city_density_level']].isnull().sum())

/var/folders/65/gwq9096x7_dbpj8wqbnt8gy00000gn/T/ipykernel_1915/2780850231.py:28: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  city_areas = df.groupby('city_id').apply(safe_city_area)


新特征的前几行：
    city_id station_name  distance_to_center  city_station_density  \
0        80        50035            0.480426            204.802689   
2        80        50035            0.480426            204.802689   
4        80        50035            0.480426            204.802689   
6        80        50035            0.480426            204.802689   
8        80        50035            0.480426            204.802689   
10       80        50035            0.480426            204.802689   
12       80        50035            0.480426            204.802689   
14       80        50035            0.480426            204.802689   
16       80        50035            0.480426            204.802689   
18       80        50035            0.480426            204.802689   

    city_density_level  
0                    1  
2                    1  
4                    1  
6                    1  
8                    1  
10                   1  
12                   1  
14                  

In [17]:
# 1. 计算每个充电站的连接器数量
connectors_per_station = df.groupby('station_name')['connector_id'].nunique()
df['station_connector_count'] = df['station_name'].map(connectors_per_station)

# 2. 计算每个充电站的平均最大充电率
avg_max_chargerate = df.groupby('station_name')['max_chargerate'].mean()
df['station_avg_max_chargerate'] = df['station_name'].map(avg_max_chargerate)

# 3. 查看结果
print("新特征的前几行：")
print(df[['station_name', 'station_connector_count', 'station_avg_max_chargerate']].head(10))

print("\n新特征的统计信息：")
print(df[['station_connector_count', 'station_avg_max_chargerate']].describe())

# 4. 检查空值
print("\n检查新特征的空值：")
print(df[['station_connector_count', 'station_avg_max_chargerate']].isnull().sum())

新特征的前几行：
   station_name  station_connector_count  station_avg_max_chargerate
0         50035                        2                         7.0
2         50035                        2                         7.0
4         50035                        2                         7.0
6         50035                        2                         7.0
8         50035                        2                         7.0
10        50035                        2                         7.0
12        50035                        2                         7.0
14        50035                        2                         7.0
16        50035                        2                         7.0
18        50035                        2                         7.0

新特征的统计信息：
       station_connector_count  station_avg_max_chargerate
count             2.991343e+06                2.991343e+06
mean              2.389166e+00                2.324927e+01
std               1.330606e+00              

In [18]:
import pandas as pd
import numpy as np

# 假设 df 是你的主数据框
# 如果 df 还没有被定义，你需要先读取你的数据
# 例如：df = pd.read_csv('your_data.csv', parse_dates=['datetime'])

# 确保 datetime 列是 datetime 类型
df['datetime'] = pd.to_datetime(df['datetime'])

# 按充电站和时间排序
df = df.sort_values(['connector_unique_id', 'datetime'])

# 1. 计算前一天同一时间的可用性
df['availability_24h_ago'] = df.groupby('connector_unique_id')['is_available'].shift(24)

# 2. 计算前一周同一时间的可用性
df['availability_1week_ago'] = df.groupby('connector_unique_id')['is_available'].shift(24 * 7)

# 3. 用当前的 is_available 值填充空值
df['availability_24h_ago'] = df['availability_24h_ago'].fillna(df['is_available'])
df['availability_1week_ago'] = df['availability_1week_ago'].fillna(df['is_available'])

# 4. 查看结果
print("新特征的前几行：")
print(df[['connector_unique_id', 'datetime', 'is_available', 'availability_24h_ago', 'availability_1week_ago']].head(20))

print("\n新特征的统计信息：")
print(df[['is_available', 'availability_24h_ago', 'availability_1week_ago']].describe())

# 5. 检查空值（应该都是0了）
print("\n检查新特征的空值：")
print(df[['availability_24h_ago', 'availability_1week_ago']].isnull().sum())

# 6. 可选：检查填充后的值是否与 is_available 相同的比例
print("\n24小时前可用性与当前可用性相同的比例：")
print((df['availability_24h_ago'] == df['is_available']).mean())

print("\n一周前可用性与当前可用性相同的比例：")
print((df['availability_1week_ago'] == df['is_available']).mean())

新特征的前几行：
   connector_unique_id            datetime  is_available  \
0              50035_1 2024-06-16 01:00:00             0   
2              50035_1 2024-06-16 02:00:00             0   
4              50035_1 2024-06-16 03:00:00             0   
6              50035_1 2024-06-16 04:00:00             0   
8              50035_1 2024-06-16 05:00:00             0   
10             50035_1 2024-06-16 06:00:00             0   
12             50035_1 2024-06-16 07:00:00             0   
14             50035_1 2024-06-16 08:00:00             0   
16             50035_1 2024-06-16 09:00:00             0   
18             50035_1 2024-06-16 10:00:00             0   
20             50035_1 2024-06-16 11:00:00             0   
22             50035_1 2024-06-16 12:00:00             0   
24             50035_1 2024-06-16 14:00:00             0   
26             50035_1 2024-06-16 15:00:00             0   
28             50035_1 2024-06-16 16:00:00             0   
30             50035_1 2024-06-

In [19]:
import numpy as np
from sklearn.neighbors import BallTree

def calculate_unique_station_density(df, radius_km=5):
    # 首先，我们需要获取唯一的充电站位置
    unique_stations = df.drop_duplicates(subset=['station_name', 'coordinates_x', 'coordinates_y'])
    
    # 将经纬度转换为弧度
    earth_radius = 6371  # 地球半径，单位为公里
    lat_rad = np.radians(unique_stations['coordinates_y'])
    lon_rad = np.radians(unique_stations['coordinates_x'])
    
    # 创建BallTree
    coords_rad = np.column_stack((lat_rad, lon_rad))
    tree = BallTree(coords_rad, metric='haversine')
    
    # 计算给定半径内的邻居数量
    radius_rad = radius_km / earth_radius
    counts = tree.query_radius(coords_rad, r=radius_rad, count_only=True)
    
    # 创建一个字典，将密度值映射到每个唯一的station_id
    density_dict = dict(zip(unique_stations['station_name'], counts - 1))
    
    # 将密度值映射回原始DataFrame
    return df['station_name'].map(density_dict)

# 使用函数计算密度
df['station_density_10km'] = calculate_unique_station_density(df, radius_km=10)
df['station_density_1km'] = calculate_unique_station_density(df, radius_km=1)
df['station_density_20km'] = calculate_unique_station_density(df, radius_km=20)

# 打印一些统计信息
print(df[['station_density_1km', 'station_density_10km', 'station_density_20km']].describe())

       station_density_1km  station_density_10km  station_density_20km
count         2.991343e+06          2.991343e+06          2.991343e+06
mean          8.647935e+00          8.843208e+01          2.055853e+02
std           1.094270e+01          8.864452e+01          1.815311e+02
min           0.000000e+00          0.000000e+00          0.000000e+00
25%           2.000000e+00          1.300000e+01          3.300000e+01
50%           5.000000e+00          5.700000e+01          1.570000e+02
75%           1.100000e+01          1.360000e+02          3.310000e+02
max           7.100000e+01          3.210000e+02          5.810000e+02


In [20]:
# df = df.drop(['station_name', 'connector_id', 'postcode','plug_type','connector_type','datetime','weather','connector_unique_id'], axis=1)
df = df.drop(['postcode','plug_type','connector_type','weather'], axis=1)
df['availability_change'] = df['is_available'] - df['availability_24h_ago']

In [21]:
# 确保 'datetime' 列是 datetime 类型
df['datetime'] = pd.to_datetime(df['datetime'])

# 找到最早的日期
earliest_date = df['datetime'].min()

df['relative_days'] = (df['datetime'] - earliest_date).dt.days

# 显示新列的一些基本统计信息
print(df['relative_days'].describe())

# 检查新列是否已正确添加
print(df[['datetime', 'relative_days']].head())
print(df[['datetime', 'relative_days']].tail())

df = df.drop('datetime', axis=1)

count    2.991343e+06
mean     1.033212e+01
std      6.354968e+00
min      0.000000e+00
25%      5.000000e+00
50%      1.000000e+01
75%      1.600000e+01
max      2.100000e+01
Name: relative_days, dtype: float64
             datetime  relative_days
0 2024-06-16 01:00:00              0
2 2024-06-16 02:00:00              0
4 2024-06-16 03:00:00              0
6 2024-06-16 04:00:00              0
8 2024-06-16 05:00:00              0
                   datetime  relative_days
2991338 2024-07-07 16:00:00             21
2991339 2024-07-07 17:00:00             21
2991340 2024-07-07 18:00:00             21
2991341 2024-07-07 19:00:00             21
2991342 2024-07-07 20:00:00             21


In [22]:
cleaned_df = df
# 显示处理后的前几行数据
print(cleaned_df.head())

# 显示DataFrame的基本信息
print(cleaned_df.info())


   city_id station_name connector_id  coordinates_x  coordinates_y  \
0       80        50035            1       -4.61764       57.21494   
2       80        50035            1       -4.61764       57.21494   
4       80        50035            1       -4.61764       57.21494   
6       80        50035            1       -4.61764       57.21494   
8       80        50035            1       -4.61764       57.21494   

   tariff_amount  tariff_connectionfee  max_chargerate  is_available  hour  \
0           0.75                   1.5               7             0     1   
2           0.75                   1.5               7             0     2   
4           0.75                   1.5               7             0     3   
6           0.75                   1.5               7             0     4   
8           0.75                   1.5               7             0     5   

   ...  city_density_level  station_connector_count  \
0  ...                   1                        2   


In [25]:
import os
from datetime import datetime

# 创建输出文件名
output_filename = "cleaned_charging_station_data.csv"

# 将数据框输出到CSV文件
df.to_csv(output_filename, index=False)

# 获取当前目录的完整路径
current_dir = os.getcwd()

# 构建完整的输出路径
output_path = os.path.join(current_dir, output_filename)

print(f"数据已成功输出到文件：{output_path}")

数据已成功输出到文件：/Users/yiming/code/evcharger-anomaly-detection/cleaned_charging_station_data.csv
